In [42]:
# Import Dependencies
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests

In [43]:
# API url 
url="http://127.0.0.1:5000/api/v1.0/movies_list"

In [44]:
# Call the api and retrieve data
response=requests.get(url)

In [45]:
# Checking if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a pandas DataFrame
    movies_combined_df = pd.DataFrame(response.json())
else:
    print("Failed to retrieve data:", response.status_code)

In [46]:
# Checking results of data requeset
movies_combined_df.head()

,age,age_desc,cleaned_genres,gender,movieId,occ_desc,occupation,poster_path,rating,timestamp,title,userId,zipcode
0,18,18-24,Action|Crime|Drama|Thriller,F,949,college/grad student,4,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,5.0,956688825,Heat,242,53706
1,18,18-24,Action|Crime|Drama|Thriller,M,949,K-12 student,10,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,4.0,942345464,Heat,363,55419
2,50,50-55,Action|Crime|Drama|Thriller,M,949,technician/engineer,17,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,4.5,1133735550,Heat,452,55117
3,25,25-34,Action|Crime|Drama|Thriller,M,949,writer,20,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,2.0,955092697,Heat,232,55408
4,45,45-49,Action|Crime|Drama|Thriller,M,949,sales/marketing,14,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,3.0,879503053,Heat,537,07704


In [47]:
# Check the number of ratings, unique movieId's, unique users, and average ratings per user and movie.
n_ratings = len(movies_combined_df)
n_movies = len(movies_combined_df['movieId'].unique())
n_users = len(movies_combined_df['userId'].unique())
 
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 43000
Number of unique movieId's: 2615
Number of unique users: 671
Average ratings per user: 64.08
Average ratings per movie: 16.44


In [48]:
# Select columns and drop duplicates
movies_df = movies_combined_df[['movieId', 'title', 'cleaned_genres']].copy()
movies_df = movies_df.drop_duplicates()
movies_df.head()

,movieId,title,cleaned_genres
0,949,Heat,Action|Crime|Drama|Thriller
5,710,GoldenEye,Adventure|Action|Thriller
6,1408,Cutthroat Island,Action|Adventure
58,524,Casino,Drama|Crime
87,4584,Sense and Sensibility,Drama|Romance


In [49]:
# Check the number of ratings and unique movieId's
n_ratings = len(movies_df)
n_movies = len(movies_df['movieId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")


Number of ratings: 2615
Number of unique movieId's: 2615


In [50]:
# Select columns
ratings_df = movies_combined_df[['userId', 'movieId', 'rating']].copy()
ratings_df.head()

,userId,movieId,rating
0,242,949,5.0
1,363,949,4.0
2,452,949,4.5
3,232,949,2.0
4,537,949,3.0


In [51]:
# Check the number of ratings, unique movieId's, and unique users
n_ratings = len(ratings_df)
n_movies = len(ratings_df['movieId'].unique())
n_users = len(ratings_df['userId'].unique())
 
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")


Number of ratings: 43000
Number of unique movieId's: 2615
Number of unique users: 671


In [52]:
# Select columns and drop duplicates
users_df = movies_combined_df[['userId', 'gender', 'zipcode', 'age_desc', 'occ_desc']] .copy()
users_df = users_df.drop_duplicates()
users_df.head()

,userId,gender,zipcode,age_desc,occ_desc
0,242,F,53706,18-24,college/grad student
1,363,M,55419,18-24,K-12 student
2,452,M,55117,50-55,technician/engineer
3,232,M,55408,25-34,writer
4,537,M,07704,45-49,sales/marketing


In [53]:
# Check the number of ratings and unique users
n_ratings = len(users_df)
n_users = len(users_df['userId'].unique())
 
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique users: {n_users}")


Number of ratings: 671
Number of unique users: 671


In [54]:
# Training set randomized options
import random
import math
RNG_SEED = 142
random_ratings = ratings_df.sample(frac=1, random_state=RNG_SEED)

# Randomize the dataframes
users = random_ratings['userId'].values
movies = random_ratings['movieId'].values
ratings = random_ratings['rating'].values

print(f"users:", users, ', shape =', users.shape)
print(f"movies:", movies, ', shape =', movies.shape)
print(f"ratings:", ratings, ', shape =', ratings.shape)

users: [575   8 211 ... 481   2 182] , shape = (43000,)
movies: [ 4857 33166   605 ... 41566   537   300] , shape = (43000,)
ratings: [3.  4.5 4.  ... 3.  4.  3. ] , shape = (43000,)


In [55]:
import tensorflow as tf
class CFModel(tf.keras.Model):
    def __init__(self, n_users, m_items, k_factors):
        super(CFModel, self).__init__()
        
        self.P = tf.keras.Sequential([
            tf.keras.layers.Embedding(n_users, k_factors, input_length=1),
            tf.keras.layers.Reshape((k_factors,))
        ])
        
        self.Q = tf.keras.Sequential([
            tf.keras.layers.Embedding(m_items, k_factors, input_length=1),
            tf.keras.layers.Reshape((k_factors,))
        ])
        
    def call(self, inputs):
        user_id, item_id = inputs
        user_latent = self.P(user_id)
        item_latent = self.Q(item_id)
        return tf.reduce_sum(tf.multiply(user_latent, item_latent), axis=1)
    
    def rate(self, user_id, item_id):
        user_embedding = self.P(tf.constant([user_id]))
        item_embedding = self.Q(tf.constant([item_id]))
        prediction = tf.reduce_sum(tf.multiply(user_embedding, item_embedding), axis=1)[0]
        return prediction.numpy()  


In [56]:
# Capture the max userId and movieId
user_id_max = ratings_df['userId'].drop_duplicates().max()
movie_id_max = ratings_df['movieId'].drop_duplicates().max()

In [57]:
# Ensure user_id and movie_id fall within range
n_users = user_id_max + 1
m_items = movie_id_max + 1

In [58]:
# Test with constant 
FACTORS = 100

In [59]:
# Colabritive filtering model
cf_model = CFModel(n_users, m_items, FACTORS)

# Compile, loss: Mean Squared Error, opimizer: Adamax
cf_model.compile(loss='mse', optimizer='adamax')

In [60]:
# Ensure compatibility with model
print("Max User ID:", user_id_max)
print("Max Movie ID:", movie_id_max)
print("Number of users (n_users):", n_users)
print("Number of items (m_items):", m_items)

Max User ID: 671
Max Movie ID: 160718
Number of users (n_users): 672
Number of items (m_items): 160719


In [61]:
# Train the model
# Set callbacks to monitor validation loss and save the best model weights
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
]

# Use epochs for training
epochs = 60

# Fit the model with callbacks
results = cf_model.fit(
    x=[users, movies],
    y=ratings,
    epochs=epochs,
    validation_split=.1,
    verbose=2,
    callbacks=callbacks
)

# Save the entire model using the TensorFlow SavedModel format
# tf.keras.models.save_model(cf_model, 'saved_model')


1210/1210 - 111s - loss: 13.7499 - val_loss: 13.7892 - 111s/epoch - 92ms/step


In [62]:
# Show the best validation RMSE
val_losses = results.history['val_loss']
min_val_loss = min(val_losses)
idx = val_losses.index(min_val_loss) + 1  # Add 1 to get epoch number starting from 1
print('Minimum RMSE at epoch', idx, '=', '{:.4f}'.format(math.sqrt(min_val_loss)))

Minimum RMSE at epoch 1 = 3.7134


In [63]:
# Select user
USER = 660

In [64]:
users_df[users_df['userId'] == USER]

,userId,gender,zipcode,age_desc,occ_desc
3511,660,M,70507,45-49,self-employed


In [65]:
# Define predicted rating
def predict_rating(userId, movieId):
    return cf_model.rate(userId - 1, movieId - 1)

In [66]:
# Get the top rated movies by current user
user_ratings = ratings_df[ratings_df["userId"] == USER][['userId', 'movieId', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(USER, x['movieId']), axis=1)

# Remove duplicate movie entries from movies_df DataFrame
unique_movies_df = movies_df.drop_duplicates(subset=['movieId'])

# Merge the user_ratings DataFrame with the unique_movies_df DataFrame to get the top 10 movies by current user.
merged_data = user_ratings.merge(unique_movies_df, on='movieId', how='inner')

# Sort the DataFrame by rating in descending order and reset the index.
top_10_rated_movies = merged_data.sort_values(by='rating', ascending=False)
top_10_rated_movies = top_10_rated_movies.reset_index(drop=True)

# Display the top 10 rated movies by current user.
top_10_rated_movies.head(10)


,userId,movieId,rating,prediction,title,cleaned_genres
0,660,260,4.5,-0.001070,The 39 Steps,Action|Thriller|Mystery
1,660,40815,4.5,0.004165,On Guard,Drama|Adventure
2,660,81847,4.5,-0.002237,The Dawn Patrol,Action|War|Drama
3,660,111759,4.5,0.007870,Don Q Son of Zorro,Western|Adventure|Romance
4,660,54001,4.5,0.003208,The Traveler,Drama
5,660,8970,4.0,0.006723,The Out-of-Towners,Comedy
6,660,4993,4.0,0.002107,5 Card Stud,Action|Western|Thriller
7,660,8961,4.0,0.001916,Bad Boys II,Adventure|Action|Comedy|Thriller|Crime
8,660,30707,4.0,0.004026,Star 80,Drama
9,660,77561,4.0,-0.001617,EVA,Science Fiction


In [70]:
# Get unrated movies for the user
recommendations = ratings_df[ratings_df['movieId'].isin(user_ratings['movieId'])== False][['movieId']].drop_duplicates()

# Generate predictions for unrated movies
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(USER, x['movieId']), axis=1)

# Merge predictions with movie information
recommended_movies = recommendations.sort_values(by='prediction', ascending=False).merge(movies_df,
                                                                                         on='movieId',
                                                                                         how='inner',
                                                                                         suffixes=['_u', '_m'])

# Filter out duplicate titles for one specific user
recommended_movies = recommended_movies.drop_duplicates(subset=['title'])

# Reset index
top_10_recommended_movies = recommended_movies.reset_index(drop=True)

# Display the top 20 recommended movies
list(top_10_recommended_movies['movieId'].head(10).values)

[640, 331, 2125, 1947, 85, 3483, 49688, 6935, 559, 3057]